In [10]:
import pandas as pd
import numpy as np
import json
import netpixi
from netpixi.integration.gt import *

# Processando dado

In [2]:
df = pd.read_csv('etf_funds/ETFs.csv')
df.info
df.columns

Index(['fund_symbol', 'quote_type', 'region', 'fund_short_name',
       'fund_long_name', 'currency', 'fund_category', 'fund_family',
       'exchange_code', 'exchange_name',
       ...
       'fund_stdev_5years', 'fund_sharpe_ratio_5years',
       'fund_treynor_ratio_5years', 'fund_alpha_10years', 'fund_beta_10years',
       'fund_mean_annual_return_10years', 'fund_r_squared_10years',
       'fund_stdev_10years', 'fund_sharpe_ratio_10years',
       'fund_treynor_ratio_10years'],
      dtype='object', length=142)

In [3]:
top10 = df[['fund_symbol','top10_holdings']]
top10

,fund_symbol,top10_holdings
0,AAAU,"""Cayman Real Assets Fund Ltd."": 0.1098, CCI (""..."
1,AADR,"AZN. L (""AstraZeneca PLC ADR""): 0.0585, MDT (""..."
2,AAXJ,"AVBNX (""Avantis© Core Fixed Income G""): 0.2708..."
3,ABEQ,"AMZN (""Amazon.com Inc""): 0.1102, MSFT (""Micros..."
4,ACES,"AVBNX (""Avantis© Core Fixed Income G""): 0.2708..."
...,...,...
2305,EPP,"SPY (""SPDR© S&P 500 ETF""): 0.0891, ""Option on ..."
2306,ERUS,"MSFT (""Microsoft Corp""): 0.0748, ""Nuance Commu..."
2307,EWA,"MSFT (""Microsoft Corp""): 0.0436, FIGXX (""Fidel..."
2308,EWC,"MSFT (""Microsoft Corp""): 0.095, AMZN (""Amazon...."


In [4]:
funds = {}
unique_assets = []
top10.dropna(subset=['top10_holdings'], inplace=True)

def parser_sampas(ticker, top10_holdings):
    assets = {}

    pairs = top10_holdings.split(',')

    for pair in pairs:
      try:
        asset, value = pair.split(':')


        asset = asset.strip()
        asset = asset.replace('"','')

        assets[asset] = float(value)

        unique_assets.append(asset)

      except Exception as e:
        pass
    
    funds[ticker] = assets


top10.apply(lambda x: parser_sampas(x['fund_symbol'], x['top10_holdings']), axis=1)
unique_assets = list(set(unique_assets))

/tmp/ipykernel_21/426082407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10.dropna(subset=['top10_holdings'], inplace=True)


In [5]:
top10['top10_holdings']

0       "Cayman Real Assets Fund Ltd.": 0.1098, CCI ("...
1       AZN. L ("AstraZeneca PLC ADR"): 0.0585, MDT ("...
2       AVBNX ("Avantis© Core Fixed Income G"): 0.2708...
3       AMZN ("Amazon.com Inc"): 0.1102, MSFT ("Micros...
4       AVBNX ("Avantis© Core Fixed Income G"): 0.2708...
                              ...                        
2305    SPY ("SPDR© S&P 500 ETF"): 0.0891, "Option on ...
2306    MSFT ("Microsoft Corp"): 0.0748, "Nuance Commu...
2307    MSFT ("Microsoft Corp"): 0.0436, FIGXX ("Fidel...
2308    MSFT ("Microsoft Corp"): 0.095, AMZN ("Amazon....
2309    MSFT ("Microsoft Corp"): 0.108, AMZN ("Amazon....
Name: top10_holdings, Length: 2297, dtype: object

In [6]:
funds

{'AAAU': {'Cayman Real Assets Fund Ltd.': 0.1098,
  'CCI (Crown Castle International Corp)': 0.0258,
  'LNG (Cheniere Energy Inc)': 0.0241,
  'United States Treasury Notes 1.62%': 0.0189,
  'FCX (Freeport-McMoRan Inc)': 0.0175,
  'United States Treasury Notes 0.12%': 0.0156,
  'AMT (American Tower Corp)': 0.0146,
  'WMB (Williams Companies Inc)': 0.0144,
  'Lme Pri Alum Futr Jul21 Xlme 20210719': 0.0139},
 'AADR': {'AZN. L (AstraZeneca PLC ADR)': 0.0585,
  'MDT (Medtronic PLC)': 0.0551,
  'BMY (Bristol-Myers Squibb Company)': 0.0534,
  'PFE (Pfizer Inc)': 0.0498,
  'LLY (Eli Lilly and Co)': 0.049,
  'AVTR (Avantor Inc)': 0.0357,
  'ANTM (Anthem Inc)': 0.0349,
  'NVS (Novartis AG ADR)': 0.0341,
  'HZNP (Horizon Therapeutics PLC)': 0.0308,
  'TFX (Teleflex Inc)': 0.0272},
 'AAXJ': {'AVBNX (Avantis© Core Fixed Income G)': 0.2708,
  'AGBGX (American Century Global Bond G)': 0.1237,
  'AVUNX (Avantis© U.S. Equity G)': 0.081,
  'ACLLX (American Century NT Focused Lg Cap Val G)': 0.0649,
  'A

In [7]:
unique_assets

['CLEYX (Columbia Select Large Cap Equity Inst3)',
 'IMCD (IMCD NV)',
 'WDC (Western Digital Corp)',
 '2023',
 'Fidelity Financial Corp 5.75%',
 'Korea 10yr Bnd Fu Jun21 Xkfe 20210615',
 'MC. PA (LVMH Moet Hennessy Louis Vuitton SE)',
 'PVH (PVH Corp)',
 'CVNA (Carvana Co Class A)',
 'TJX (TJX Companies Inc)',
 'FL (Foot Locker Inc)',
 '09618 (JD.com Inc Ordinary Shares - Class A)',
 'Federal National Mortgage Association 2.14%',
 'Howmet Aerospace Inc 6.88%',
 'PEG (Public Service Enterprise Group Inc)',
 'SNPS (Synopsys Inc)',
 'Sociedad Hipotecaria Federal SNC IBD 0%',
 'LP and CVR Nitrogen Finance Corporation 6.13%',
 'PFE (Pfizer Inc)',
 'ATO (Atmos Energy Corp)',
 'Granite Point Mortgage Trust Inc 6.375%',
 'DXS. AX (Dexus)',
 'ODFL (Old Dominion Freight Line Inc Ordinary Shares)',
 'ULE. L (Ultra Electronics Holdings PLC)',
 'Kinross Gold Corporation 6.88%',
 'HAWAII ST DEPT TRANSN ARPTS DIV LEASE REV CTFS PARTN 5%',
 'TWNK (Hostess Brands Inc Class A)',
 'Wells Fargo Commercial

In [8]:
matrix = pd.DataFrame(columns=unique_assets, index=funds.keys())
matrix.fillna(0, inplace=True)

for fund in funds:
  for asset in funds[fund]:
    matrix.loc[fund, asset] = funds[fund][asset]

In [9]:
# Euclidean distance between two vectors for all funds
from scipy.spatial.distance import pdist, squareform
distances = pd.DataFrame(squareform(pdist(matrix, 'euclidean')), columns=matrix.index, index=matrix.index)

distances

,AAAU,AADR,AAXJ,ABEQ,ACES,ACIO,ACSI,ACTV,ACWI,ACWV,...,VEGI,WPS,ZIVZF,EEM,EIRL,EPP,ERUS,EWA,EWC,EWD
AAAU,0.000000,0.185358,0.362142,0.228881,0.362142,0.203304,0.362142,0.362142,0.362142,0.356020,...,0.141453,0.151881,0.191832,0.146391,0.146391,0.151881,0.169695,0.152801,0.198725,0.201292
AADR,0.185358,0.000000,0.368590,0.238952,0.368590,0.214579,0.368590,0.368590,0.368590,0.362577,...,0.157227,0.166671,0.203743,0.161685,0.161685,0.166671,0.175257,0.163321,0.210246,0.212673
AAXJ,0.362142,0.368590,0.000000,0.392284,0.000000,0.377933,0.000000,0.000000,0.000000,0.070355,...,0.348582,0.352942,0.371887,0.350615,0.350615,0.352942,0.360967,0.353339,0.375489,0.376854
ABEQ,0.228881,0.238952,0.392284,0.000000,0.392284,0.253126,0.392284,0.392284,0.392284,0.386640,...,0.206758,0.214028,0.238546,0.210168,0.210168,0.214028,0.200604,0.183983,0.114998,0.116129
ACES,0.362142,0.368590,0.000000,0.392284,0.000000,0.377933,0.000000,0.000000,0.000000,0.070355,...,0.348582,0.352942,0.371887,0.350615,0.350615,0.352942,0.360967,0.353339,0.375489,0.376854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EPP,0.151881,0.166671,0.352942,0.214028,0.352942,0.186424,0.352942,0.352942,0.352942,0.346659,...,0.115890,0.000000,0.173843,0.121869,0.121869,0.000000,0.149058,0.129498,0.181420,0.184228
ERUS,0.169695,0.175257,0.360967,0.200604,0.360967,0.201204,0.360967,0.360967,0.360967,0.354825,...,0.138417,0.149058,0.189605,0.143460,0.143460,0.149058,0.000000,0.126396,0.156302,0.153336
EWA,0.152801,0.163321,0.353339,0.183983,0.353339,0.187175,0.353339,0.353339,0.353339,0.347063,...,0.117093,0.129498,0.174647,0.123014,0.123014,0.129498,0.126396,0.000000,0.127469,0.142815
EWC,0.198725,0.210246,0.375489,0.114998,0.375489,0.226226,0.375489,0.375489,0.375489,0.369589,...,0.172785,0.181420,0.205416,0.176850,0.176850,0.181420,0.156302,0.127469,0.000000,0.092158


In [27]:
media_jrg = distances.mean().mean()

# Criando Grafo

In [22]:
g = Graph(directed=False) # não-dirigido

In [24]:
for fund in funds:
    try:
        g.add_vertex(fund)
    except:
        print(fund)

In [28]:
for row, col in distances.iterrows():
    for index, value in col.items():
        if row == index: 
            continue
            
        if value > media_jrg:
            continue
        
        try:
            g.add_edge(row, index)
        except:
            pass

In [29]:
gt_save(g, "sampa.net.gz")

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


KeyboardInterrupt: 